In [1]:
import torch
import torchvision

n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
# momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/data', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/data', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

print(batch_idx)
print(example_data.shape, example_targets.shape)

print(example_data[0].shape)
print(example_data[0][0].shape)
print(example_data[1].shape)

0
torch.Size([1000, 1, 28, 28]) torch.Size([1000])
torch.Size([1, 28, 28])
torch.Size([28, 28])
torch.Size([1, 28, 28])


In [2]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.tight_layout()
    plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
    plt.title("Ground Truth: {}".format(example_targets[i]))
    plt.xticks([])
    plt.yticks([])
fig

<Figure size 640x480 with 6 Axes>

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        # layer 1
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)#, stride=1, padding=2)
        self.conv1_batch = nn.BatchNorm2d(20)
        
        # layer 2
        self.conv2 = nn.Conv2d(20, 20,kernel_size=5)#, stride=1, padding=2)
        self.conv2_batch = nn.BatchNorm2d(20)
        
        # layer 3
        self.fc1 = nn.Linear(20* 20 * 20, 50)
        self.fc1_batch = nn.BatchNorm1d(50)
        
        # layer 4
        self.fc2 = nn.Linear(50, 10)
        self.fc2_batch = nn.BatchNorm1d(10)

    def forward(self, x):
        
        # layer 1
        x = self.conv1(x)
#         x = F.max_pool2d(x, 2)
        x = nn.functional.sigmoid(x)
        x = self.conv1_batch(x)
#         print("1: ", x.shape)
        
        # layer 2
        x = self.conv2(x)
#         x = F.max_pool2d(x, 2)
        x = nn.functional.sigmoid(x)
        x = self.conv2_batch(x)
#         print("2: ", x.shape)
        
        # layer 3
        x = x.view(-1, 20* 20 * 20)
        x = self.fc1(x)
        x = self.fc1_batch(x)
#         print("3: ", x.shape)
        
        # layer 4
        x = self.fc2(x)
#         print("4: ", x.shape)
        
        return F.log_softmax(x)

In [4]:
# create network and set optimizer -> article : Adam, lr = 0.001, b1 = 0.9, b2 = 0.999
network = Network()
optimizer = optim.Adam(network.parameters(), lr = 0.001, betas = (0.9, 0.999))
loss_function = nn.MSELoss()

In [5]:
# init arrays for train/test errors
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs)]

In [6]:
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        
        output = network(data)
#         print("train: ", output.shape, target.shape)
#         loss = loss_function(output, target)
        loss = F.nll_loss(output, target)
        
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append(
            (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            torch.save(network.state_dict(), './results/model.pth')
            torch.save(optimizer.state_dict(), './results/optimizer.pth')

In [9]:
def test():
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()

            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
            
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print(test_losses)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [10]:
# test()
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()
    


D:\DISERTATIE_LUCRU\envs\data-science\lib\site-packages\torch\nn\functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
D:\DISERTATIE_LUCRU\envs\data-science\lib\site-packages\ipykernel_launcher.py:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.030895
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.021326
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.049696
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.021835
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.028267
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.073872
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.035588
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.088216
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.035216
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.112876
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.098953
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.049525
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.056173
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.022554
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.067580
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.025939
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.040352
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.069592
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.051399
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.125390
Train Epoch: 1 [12800/60000 (

D:\DISERTATIE_LUCRU\envs\data-science\lib\site-packages\torch\nn\_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


[0.05439461364746094]

Test set: Avg. loss: 0.0544, Accuracy: 9839/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.072106
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.020954
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.016838
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.010789
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.065233
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.078191
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.035725
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.035944
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.054258
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.064707
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.006317
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.015374
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.066487
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.012936
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.029678
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.017408
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.015392
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.061056
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.047137